In [ ]:
# imports and settings

import os
import time
import pickle
import warnings
import pandas as pd
import networkx as nx
from networkx.algorithms.coloring import greedy_color
import matplotlib.pyplot as plt
from copy import deepcopy

import numpy as np
from numpy import linalg as LA
from numpy import histogram2d

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt, welch, get_window
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile
from scipy.stats import wasserstein_distance_nd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, linear_kernel

import utils as ut
%load_ext autoreload
%autoreload 2

# do not show warnings
warnings.filterwarnings("ignore")

print("Imports complete.")

In [ ]:
fs, data = wavfile.read("../data/RBW6695_20260107_092700 2.wav")

In [ ]:
F, T, Sxx, phasogram = ut.calc_spectrogram(data, fs, nperseg=32000, percent_overlap=0.5, window='hanning', remove_dc=120, crop_freq=80000)
pxx = ut.calc_welch_from_spectrogram(Sxx, normalization_window_size=5)
K = ut.get_all_Ks(phasogram, F, 10, mode='wasserstein')
K = (K - np.min(K)) / (np.max(K) - np.min(K))  # normalize to [0,1]
# K = ut.get_all_Ks(phasogram, F, 10, mode='laplacian')
# K = K / np.mean(K)
# K = np.mean(K) - K
# K = ut.rw_normalization(K)
sym = ut.calc_rw_symmetry(pxx, window_size=16)

fig = make_subplots(rows=1, cols=4, column_titles=["Spectrogram", "Welch", "K", "Symmetry"], column_widths=[0.7, 0.1, 0.1, 0.1], horizontal_spacing=0.02, shared_yaxes=True)
fig.add_trace(go.Heatmap(z=Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=1)
fig.add_trace(go.Scatter(y=F, x=pxx, mode='lines', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=F, x=K, mode='lines', line=dict(color='purple'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=F, x=sym, mode='lines', line=dict(color='green'), showlegend=False), row=1, col=4)
fig.update_layout(height=600, width=1200, title_text="Spectrogram, Welch, K, and Symmetry for UAV Signal")
fig.show()

In [ ]:
def calc_rw_symmetry(pxx, window_size):
    half = window_size // 2
    vals = np.zeros(pxx.shape[0])
    for i in range(pxx.shape[0]):
        # compute a local half-window without modifying the outer 'half' value
        local_half = min(half, i, pxx.shape[0] - i - 1)
        if local_half <= 0:
            vals[i] = 0
            continue

        fw = pxx[i-local_half:i]
        bw = pxx[i+1:i+local_half+1]  #[::-1]

        # ensure equal lengths (safety)
        if fw.shape[0] != bw.shape[0]:
            minlen = min(fw.shape[0], bw.shape[0])
            fw = fw[-minlen:]
            bw = bw[:minlen]
        
        vals[i] = np.convolve(fw, bw, mode='valid')[0]
    return vals

def spectrogram_symmetry(Sxx, window_size):
    half = window_size // 2
    sym = np.zeros_like(Sxx)
    for i in range(Sxx.shape[0]):
        local_half = min(half, i, Sxx.shape[0] - i - 1)
        if local_half <= 0:
            continue

        fw = Sxx[i-local_half:i, :]
        bw = Sxx[i+1:i+local_half+1, :][::-1]

        if fw.shape[0] != bw.shape[0]:
            minlen = min(fw.shape[0], bw.shape[0])
            fw = fw[-minlen:, :]
            bw = bw[:minlen, :]

        sym[i, :] = np.sum(fw * bw, axis=0)
        # sym[i, :] = signal.correlate2d(fw, bw)
    return sym

In [ ]:
sym_spec = spectrogram_symmetry(Sxx, window_size=16)
sym_pxx = ut.calc_welch_from_spectrogram(sym_spec, normalization_window_size=5)
sym_pxx[:5] = 0
sym_pxx[-5:] = 0
fig = make_subplots(rows=2, cols=2, column_widths=[0.9, 0.1], horizontal_spacing=0.02, shared_yaxes=True)
fig.add_trace(go.Heatmap(z=Sxx, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=1, col=1)
fig.add_trace(go.Scatter(y=F, x=pxx, mode='lines', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Heatmap(z=sym_spec, x=T, y=F, colorscale='Viridis', showlegend=False, showscale=False), row=2, col=1)
fig.add_trace(go.Scatter(y=F, x=sym_pxx, mode='lines', line=dict(color='green'), showlegend=False), row=2, col=2)   
fig.update_layout(height=800, width=1200, title_text="Spectrogram and Spectrogram Symmetry for UAV Signal")
fig.show()